# In this notebook i have implemented model with logistic regresseion 

In [ ]:
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge,RidgeCV,Lasso,LassoCV,ElasticNet,ElasticNetCV,LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

Dataset has loaded from the below url path

In [ ]:
df= pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv")
df.head()

In [ ]:
df.shape

Analyzing the dataset using the profilereport()

In [ ]:
ProfileReport(df)

Handling the missing values()

In [ ]:
df['BMI'] = df['BMI'].replace(0,df['BMI'].mean())

In [ ]:
df['BloodPressure'] = df['BloodPressure'].replace(0,df['BloodPressure'].mean())

In [ ]:
df['Insulin'] = df['Insulin'].replace(0,df['Insulin'].mean())

In [ ]:
df['Glucose'] = df['Glucose'].replace(0,df['Glucose'].mean())

In [ ]:
df['SkinThickness'] = df['SkinThickness'].replace(0,df['SkinThickness'].mean())

Analyzing the dataset after handling the missing values()

In [ ]:
ProfileReport(df)

checking outlier using box plot

In [ ]:
q=df['Insulin'].quantile(.90)
df_new=df[df['Insulin']<q]

In [ ]:
df_new

In [ ]:
q=df_new['Pregnancies'].quantile(.98)
df_new=df_new[df_new['Pregnancies']<q]

q=df_new['BMI'].quantile(.99)
df_new=df_new[df_new['BMI']<q]

q=df['SkinThickness'].quantile(.99)
df_new=df_new[df_new['SkinThickness']<q]

q=df_new['Insulin'].quantile(.98)
df_new=df_new[df_new['Insulin']<q]

In [ ]:
df_new

In [ ]:
ProfileReport(df_new)

Splitting the dataset in dependent and independent features

In [ ]:
y=df_new['Outcome']
y

In [ ]:
x=df_new.drop(columns=['Outcome'])
x

In [ ]:
x.columns

Performing standard scalar

Data has scaled using StandardScaler

In [ ]:
scalar=StandardScaler()
x_scaled=scalar.fit_transform(x)
x_scaled

In [ ]:
x_scaled_df=pd.DataFrame(x_scaled)
x_scaled_df.head()

In [ ]:
x_scaled_df.describe()

In [ ]:
y

calculating VIF for all the independent features

Checking the Multi co-linearity between the independent features using VIF

In [ ]:
vif_df=pd.DataFrame()
vif_df['features']=x.columns
vif_df['vif_score']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif_df

Splitting the dataset using train test split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.20, random_state=144)

In [ ]:
x_train

In [ ]:
x_test

Applying the logistic regression algorithm

In [ ]:
lor= LogisticRegression(verbose=1)

In [ ]:
lor.fit(x_train,y_train)

In [ ]:
lor.predict([[-0.86724406,  1.72932397,  0.18273346, -1.00048758,  1.61961985,
       -1.30243789, -0.0135205 ,  0.00209362]])

In [ ]:
lor.predict_proba([[-0.86724406,  1.72932397,  0.18273346, -1.00048758,  1.61961985,
       -1.30243789, -0.0135205 ,  0.00209362]])

In [ ]:
lor.predict_log_proba([[-0.86724406,  1.72932397,  0.18273346, -1.00048758,  1.61961985,
       -1.30243789, -0.0135205 ,  0.00209362]])

In [ ]:
y_test.iloc[0]

Training the model again by changing the solver, you can use different solver to train model

In [ ]:
new_lor=LogisticRegression(solver='liblinear',verbose=1)
new_lor.fit(x_train,y_train)

In [ ]:
new_lor.predict(x_train[[2]])

In [ ]:
y_test.iloc[0]

In [ ]:
lor,new_lor

In [ ]:
y_pred = lor.predict(x_test)
y_pred

In [ ]:
y_pred_liblinear= new_lor.predict(x_test)
y_pred_liblinear

Model selection using confusion matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred_liblinear)

Checking accuracy, recall, precision, specificity and f1 call of the both model

In [ ]:
def model_evaluation(a,b):
    tn,tp,fn,fp = confusion_matrix(y_test,y_pred).ravel()
    accuracy = (tp+tn)/(tp+fn+fp+tn)
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+tn)
    specificity = (tn)/(fp+tn)
    f1_score = 2*(recall*precision)/(recall-precision)
    result = {'accuracy':accuracy,'precision':precision,'recall':recall,'specificity':specificity,'f1_score':f1_score}
    return result
model_evaluation(y_test,y_pred)

In [ ]:
model_evaluation(y_test,y_pred_liblinear)

AUC and ROC 

In [ ]:
auc=roc_auc_score(y_test,y_pred)
auc

In [ ]:
auc_libc=roc_auc_score(y_test,y_pred_liblinear)
auc_libc

In [ ]:
roc_curve(y_test,y_pred)

In [ ]:
roc_curve(y_test,y_pred_liblinear)

In [ ]:
# plt.plot(fpr,tpr,color='orange',label="")
# plt.plot([0,1],[0,1],color='darkblue',linestyle='--',label='ROC curve (area = %0.2f)' % auc)
# plt.xlabel('false positive error')
# plt.ylabel('True positive error')
# plt.title('Recevicer operation chara (ROC) Curve')
# plt.legend()
# plt.show()

The model is giving 78% accuracy in testing data

In [ ]:
lor.score(x_test,y_test),new_lor.score(x_test,y_test)